In [33]:
import dotenv
import os
from requests import Session

dotenv.load_dotenv()

API_KEY = os.getenv("COINMARKETCAP_API_KEY")
API_ENDPOINT = "https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest"
COINS = "BTC,ETH"

s = Session()

response_api = s.get(
    url=API_ENDPOINT,
    headers={"Accepts": "application/json", "X-CMC_PRO_API_KEY": API_KEY},
    params={"symbol": COINS},
)

data = response_api.json()

In [34]:
from dataclasses import dataclass
import pandas as pd


@dataclass
class CoinData:
    kafka_key: str
    kafka_value: dict


data = response_api.json().get("data", {})

coins_data = [
    CoinData(
        kafka_key=k,
        kafka_value=next(iter(data.get(k, {}))),
    )
    for k in data.keys()
]

for quote in coins_data:
    data: dict = quote.kafka_value

    data.pop("tags", [])
    df = pd.DataFrame([data.pop("quote", {}).pop("USD", {}) | data])

    df = df.T.sort_index()

    display(df.head(100))

,0
circulating_supply,19823646
cmc_rank,1
date_added,2010-07-13T00:00:00.000Z
fully_diluted_market_cap,2013761181198.699951
id,1
infinite_supply,False
is_active,1
is_fiat,0
last_updated,2025-02-13T12:30:00.000Z
market_cap,1900956608791.665039


,0
circulating_supply,120545397.973535
cmc_rank,2
date_added,2015-08-07T00:00:00.000Z
fully_diluted_market_cap,321189702219.299988
id,1027
infinite_supply,True
is_active,1
is_fiat,0
last_updated,2025-02-13T12:30:00.000Z
market_cap,321189702219.297546


In [35]:
from pprint import pprint


@dataclass
class CoinMetrics:
    symbol: str
    name: str
    last_updated: str
    price: float  # USD
    percent_change_1h: float
    percent_change_24h: float
    percent_change_7d: float
    percent_change_30d: float
    percent_change_60d: float
    percent_change_90d: float
    volume_24h: float  # USD
    volume_change_24h: float  # %
    circulating_supply: float  # cryptocurrency
    total_supply: float  # cryptocurrency
    max_supply: float  # cryptocurrency
    market_cap: float  # USD
    market_cap_dominance: float  # %


data = response_api.json().get("data", {})

coins_data = [
    CoinData(
        kafka_key=k,
        kafka_value=next(iter(data.get(k, {}))),
    )
    for k in data.keys()
]

coins_metrics = []
for coin in coins_data:
    data = coin.kafka_value
    quote = data.get("quote", {}).get("USD", {})

    metrics = CoinMetrics(
        symbol=data.get("symbol"),
        name=data.get("name"),
        last_updated=quote.get("last_updated"),
        price=quote.get("price"),
        percent_change_1h=quote.get("percent_change_1h"),
        percent_change_24h=quote.get("percent_change_24h"),
        percent_change_7d=quote.get("percent_change_7d"),
        percent_change_30d=quote.get("percent_change_30d"),
        percent_change_60d=quote.get("percent_change_60d"),
        percent_change_90d=quote.get("percent_change_90d"),
        volume_24h=quote.get("volume_24h"),
        volume_change_24h=quote.get("volume_change_24h"),
        circulating_supply=data.get("circulating_supply"),
        total_supply=data.get("total_supply"),
        max_supply=data.get("max_supply"),
        market_cap=quote.get("market_cap"),
        market_cap_dominance=quote.get("market_cap_dominance"),
    )

    coins_metrics.append(metrics)

pprint(coins_metrics)

[CoinMetrics(symbol='BTC',
             name='Bitcoin',
             last_updated='2025-02-13T12:30:00.000Z',
             price=95893.38958089067,
             percent_change_1h=-0.2802357,
             percent_change_24h=-0.32154013,
             percent_change_7d=-2.80156435,
             percent_change_30d=-0.24933714,
             percent_change_60d=-6.69021601,
             percent_change_90d=6.30054643,
             volume_24h=51164506898.55355,
             volume_change_24h=36.6743,
             circulating_supply=19823646,
             total_supply=19823646,
             max_supply=21000000,
             market_cap=1900956608791.665,
             market_cap_dominance=59.9336),
 CoinMetrics(symbol='ETH',
             name='Ethereum',
             last_updated='2025-02-13T12:30:00.000Z',
             price=2664.470876688402,
             percent_change_1h=-0.18630392,
             percent_change_24h=1.26719088,
             percent_change_7d=-4.68274998,
             percent_ch

## fields of interest
- price USD
- market cap USD
- volume 24h, volume change 24h %
- trading velocity (volume 24h/market cap) %
- % change: 1h, 24h, 7d, 30d, 90d
- total supply, max supply